#**TODO**

Refine keywords : strip \n, ", ' etc ...

Handle ill-encoded keywords : utf-8 ...?

#**Import Modules**

In [0]:
import openpyxl

In [58]:
!pip install xlsxwriter
import xlsxwriter

In [0]:
import pandas as pd

In [0]:
import numpy as np

In [0]:
import gc

In [0]:
from os import listdir
from os.path import isfile, join

In [0]:
from google.colab import files

#**Mount Google Drive**

In [64]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [65]:
cd drive/My Drive/Colab Notebooks/Paper/Telecommunication

[Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks/Paper/Telecommunication'
/content/drive/My Drive/Colab Notebooks/Paper/Telecommunication


In [66]:
ls

Crawler/  Data/  Paper-Keywords/


#**Read and Process Data**

In [0]:
def pair_to_vector(idx, length):
    res = [0] * length
    for i in idx:
        res[i] = 1
    return res

In [0]:
DATAPATH = "./Data/raw/Korean/"

In [68]:
# fetch journal names
journals = [f for f in listdir(DATAPATH) if isfile(join(DATAPATH, f))]

for i in range(len(journals)):
    journals[i] = journals[i][:-5]

journals

['korean_telecommunications_policy_review', 'journal_of_korea_trade']

In [0]:
keywords = []
papers = []
keyword_paper_pair = []
curr_idx = -1

for i in range(len(journals)):
    # open workbook
    wb = openpyxl.load_workbook(DATAPATH + journals[i] + ".xlsx")
    ws = wb.active
    MAXROW = ws.max_row
    MAXCOL = ws.max_column
    curr = 0

    for r in ws.rows:
        # ignore header
        if(r[0].row == 1):
            continue
        # halt loop
        if(r[0].row > MAXROW):
            break
        # update cursor
        if(isinstance(r[2].value, str)):
            curr = r[2].value
            papers.append(curr)
            curr_idx += 1

        # row corresponds to a keyword
        if(isinstance(r[3].value, str)):
            # duplicate exists
            if(r[3].value.lower() in keywords):
                keyword_paper_pair[keywords.index(r[3].value.lower())].append(curr_idx)
            # duplicate doesn't exist
            else:
                keywords.append(r[3].value.lower())
                keyword_paper_pair.append([curr_idx])
    
# encode keyword-paper pair to vector
keyword_vector = []
for i in range(len(keyword_paper_pair)):
    keyword_vector.append(pair_to_vector(keyword_paper_pair[i], curr_idx + 1))

wb.close()

In [70]:
len(papers)

613

In [71]:
len(keywords)

2208

In [72]:
len(keyword_vector)

2208

#**Write Paper-Keyword Matrix to Excel**

In [0]:
def initial(s):
    res = ''
    for a in s.split(' '):
        res += a[0]
    return res

In [0]:
TARGETPATH = "./Data/mat/Korean/"

In [0]:
RESFILENAME = ""
for journal in journals:
    RESFILENAME += "_" + initial(journal.replace("_", " ").upper())
RESFILENAME = RESFILENAME[1:]

wb = xlsxwriter.Workbook(TARGETPATH + RESFILENAME + "_mat.xlsx")
ws = wb.add_worksheet()

for i in range(len(papers)):
    ws.write(i + 1, 0, papers[i])
for j in range(len(keyword_vector)):
    ws.write(0, j + 1, keywords[j])
    for i in range(len(keyword_vector[j])):
        ws.write(i + 1, j + 1, keyword_vector[j][i])

wb.close()

#**Sort Keywords by Frequency**

**Make a Dataframe**

In [0]:
df = pd.DataFrame(keyword_vector)
df = df.transpose()
df.index = papers
df.columns = keywords

In [0]:
df

,agricultural trade,gravity,trade costs,discrete choice,productivity,comparative advantage,demand complementarities,goods & services,oligopoly,firm-level exports,quality,banking crises,china,financial frictions,lender of last resort,agglomeration,assignment models,cities,inflation,impact of globalization,strategic interactions,market structure,phillips curve,resource insecurity,interstate disputes,conflict,arming,trade openness,sovereign default,corporate borrowing,interest rate spreads,fiscal policy,multinational production,non-neutral technologies,labor shares,non-homothetic preferences,skill premium,per capita income,structural change,international trade,...,ngo program,education management,nonformal,low-cost,technological capability,technological learning,technological complexity,farm equipment,carbon dioxide emission,greenhouse gas,r & d,researchers,development strategy,land resettlement,resource economics,quasi-fiscal deficits,capital adequacy,greenhouse effect,cross-national,education finance,small growers,trypanosomiasis,tsetse flies,fuelwood shortage,household fuel use,fuelwood collection,share-cropping,interlinkage model,financial reform,measurement indices,forest size,historical trends,united nations conferences,influence,unced,world food conference,stakeholder analysis,research policy,rural mechanization,income-distribution
"Technology, ecology and agricultural trade",1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
One way to the top: How services boost the demand for goods,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
A quantitative model of international lending of last resort,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
The comparative advantage of cities,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"Globalization, market structure and inflation dynamics",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"The paths to rain forest destruction: Crossnational patterns of tropical deforestation, 1975–1990",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0
Settler households and land-use patterns in the Amazon frontier: Farm-level evidence from Ecuador,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"The frontiers of influence: NGO lobbying at the 1974 World Food Conference, the 1992 Earth Summit and beyond",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0
"Stakeholders, science and decision making for poverty-focused rural mechanization research and development",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0


**Write Ranks to Excel**

In [0]:
wb = openpyxl.Workbook()
wb.save("kwd_rank_readable.xlsx")

freq_tot = df.sum(axis=0).tolist()
freq_tot_mat = []
for i in range(len(freq_tot)):
    freq_tot_mat.append([keywords[i], freq_tot[i]])
freq_tot_mat.sort(key=lambda x: x[1])
freq_tot_mat.reverse()
sheetno = 1
wb.create_sheet(str(sheetno))
ws = wb.get_sheet_by_name(str(sheetno))
rowno = 1
colno = 1
for j in range(len(freq_tot_mat)):
    if((j + 1) % 100 == 0):
        rowno = 1
        colno += 2
    if((j + 1) % 500 == 0):
        sheetno += 1
        rowno = 1
        colno = 1
        wb.create_sheet(str(sheetno))
        ws = wb.get_sheet_by_name(str(sheetno))
    ws.cell(row=rowno, column = colno).value = freq_tot_mat[j][0]
    ws.cell(row=rowno, column = colno + 1).value = freq_tot_mat[j][1]
    rowno += 1
wb.save("kwd_rank_readable.xlsx")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24:

#**Download Files**

In [0]:
files.download(TARGETPATH + RESFILENAME + "_mat.xlsx")